In [2]:
from __future__ import division
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import csv
import numpy as np

In [3]:
# Zonas_kmeans.csv, Zonas_dbscan.csv
file_name = '../Data/Zonas_dbscan.csv'

In [7]:
f = open(file_name, 'r')
csvfile = csv.reader(f)
caracteristicas = csvfile.next() #skip the headers
zonas = []

for zona in csvfile:
    zonas.append(map(int, zona))

In [8]:
min_max_scaler = MinMaxScaler()
zonas_norm = min_max_scaler.fit_transform([zona[1:] for zona in zonas])

pca_estimator = PCA(n_components = 2)
X_pca = pca_estimator.fit_transform(zonas_norm)

print 'Varianza explicada por cada varaible: ' + str(pca_estimator.explained_variance_ratio_)
print 'Varianza total explicada: ' + str(sum(pca_estimator.explained_variance_ratio_))

Varianza explicada por cada varaible: [ 0.80022241  0.1217984 ]
Varianza total explicada: 0.922020811566


In [9]:
plt.scatter([x[0] for x in X_pca], [x[1] for x in X_pca])
plt.show()

In [10]:
fig, ax = plt.subplots()
plt.xlim(-0.2, 0.55)
plt.ylim(-0.2, 1)
ax.grid(True)

N = len(X_pca)
numbers = np.arange(len(X_pca))

for i in range(N):
    plt.text(X_pca[i][0], X_pca[i][1], numbers[i])

plt.show()

In [11]:
import sklearn.neighbors
from scipy import cluster 

In [12]:
zona_peligrosa = zonas[11]
zonas = zonas[:11] + zonas[12:]
X_pca = np.concatenate((X_pca[:11], X_pca[12:]))

In [13]:
dist = sklearn.neighbors.DistanceMetric.get_metric('euclidean')
matsim = dist.pairwise(X_pca)

In [14]:
clusters = cluster.hierarchy.linkage(matsim, method = 'ward')
cluster.hierarchy.dendrogram(clusters, color_threshold=0)
plt.show()

In [15]:
cut = 6
labels = cluster.hierarchy.fcluster(clusters, cut, criterion='distance')
unique_labels = set(labels)
print 'Número de clusters %d' % (len(unique_labels) + 1)

colors = np.array(list('bgrcmykbgrcmykbgrcmykbgrcmyk'))
colors = np.hstack([colors] * 20)

fig, ax = plt.subplots()

plt.xlim(-0.2, 0.55)
plt.ylim(-0.2, 1)
ax.grid(True)

for i in range(len(zonas)):
    plt.text(X_pca[i][0], X_pca[i][1], 'x', color = colors[labels[i]])

fig.tight_layout()
plt.show()

Número de clusters 4


In [16]:
zone_groups = dict()
for i, zona in enumerate(zonas):
    if labels[i] in zone_groups:
        zone_groups[labels[i]].append(zona)
    else:
        zone_groups[labels[i]] = [zona]
zone_groups[len(unique_labels)+1] = [zona_peligrosa]

In [34]:
features = []

for key in zone_groups:
    zone_feature = []
    for i in range(1, len(zone_groups[key][0])):
        feature = [zone[i] for zone in zone_groups[key]]
        maxi = max(feature)
        mini = min(feature)
        aver = sum(feature) / len(feature)
        zone_feature.append([maxi, mini, aver])
    features.append(zone_feature)

In [36]:
#Hacer un csv con features y caracteristicas